In [12]:
import os
import pandas as pd
from functions import get_all_origins, find_pattern_for_quantity, convert_to_grams, relation_qnt_preco, remove_spaces, clean_text
import re
import numpy as np
import ast



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv("./back_words.csv")
df.info()
df["back_words"] = df["back_words"].apply(ast.literal_eval)
df["back_words_original"] = df["back_words_original"].apply(ast.literal_eval)
df["back_words_original_exclude_word"] = df["back_words_original_exclude_word"].apply(ast.literal_eval)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   back_words                        21402 non-null  object
 1   back_words_original               21402 non-null  object
 2   back_words_original_exclude_word  21402 non-null  object
 3   last_1                            21327 non-null  object
 4   last_2                            20788 non-null  object
 5   last_3                            20027 non-null  object
dtypes: object(6)
memory usage: 1003.3+ KB


In [14]:
from corpora import get_corpus

# Carregar o corpus de palavras em português
corpus = get_corpus('words-pt')

ModuleNotFoundError: No module named 'corpus'

In [2]:
personal_pronouns = ['Nós', 'Ti', 'Ele', 'Si', 'Consigo', 'Elas', 'Vós', 'Eu', 'Eles', 'Ela', 'Mim', 'Tu']
oblique_pronouns = ['Lhe', 'A', 'Contigo', 'Vos', 'Convosco', 'Lhes', 'O', 'Se', 'As', 'Nos', 'Me', 'Te', 'Os']
demonstrative_pronouns = ['Estas', 'Iste', 'Esta', 'Isso', 'Estes', 'Aquele', 'Aquilo', 'Essa', 'Aquelas', 'Este', 'Esse', 'Isto', 'Aqueles', 'Essas', 'Esses', 'Aquela']
possessive_pronouns = ['Minhas', 'Minha', 'Seu', 'Meus', 'Suas', 'Nossas', 'Sua', 'Tua', 'Vosso', 'Vossas', 'Nossa', 'Meu', 'Teus', 'Teu', 'Nossos', 'Tuas', 'Nosso', 'Seus', 'Vossa', 'Vossos']
indefinite_pronouns = ['Qualquer', 'Nenhum', 'Tanto', 'Muitos', 'Algum', 'Algo', 'Tudo', 'Muito', 'Alguém', 'Poucos', 'Todo', 'Nada', 'Cada', 'Vários', 'Cada um', 'Pouco', 'Outrem', 'Ninguém', 'Outro']
relative_pronouns = ['Quem', 'Cuja', 'Que', 'O qual', 'Qual', 'Onde', 'Cujo', 'Quanto']
interrogative_pronouns = ['Quem', 'Por que', 'O que', 'Qual', 'Qualquer coisa', 'Onde', 'Quando', 'Quanto', 'Quanto a', 'Como']
prepositions = ['Por', 'Conforme', 'Menos', 'Ante', 'Após', 'Mediant', 'Segundo', 'Desde', 'Contudo', 'Sobre', 'Entre', 'Até', 'Trás', 'Contra', 'De', 'Visto', 'Com', 'Sob', 'Sem', 'Perante', 'Para', 'A', 'Salvo', 'Exceto', 'Em', 'Durante']

BRAZIL_PRONOUNS = [personal_pronouns] + [oblique_pronouns] + [demonstrative_pronouns] + [possessive_pronouns] + [indefinite_pronouns] + [relative_pronouns] + [interrogative_pronouns] + [prepositions]

In [3]:
convert_to_lower = lambda x: [word.lower() for word in x]

terms = []
for index, row in df.iterrows():
    # Criar uma lista com os termos das três colunas
    # terms = row['last_1'], row['last_2'], row['last_3']
    
    # Iterar sobre os termos
    # for idx, term in enumerate(terms):
        # Verificar se o termo está presente em alguma lista de pronomes convertidos para minúsculas
        # flag = False
        # for pronouns_list in BRAZIL_PRONOUNS:
        #     if term in convert_to_lower(pronouns_list):
        #         print(str(idx + 1) + " " + term)
        #         flag = True    
        # if (flag):
    term_1_ = row['back_words_original_exclude_word'][-1] if len(row['back_words_original_exclude_word']) >= 1 else ""
    term_2_ = row['back_words_original_exclude_word'][-2] if len(row['back_words_original_exclude_word']) >= 2 else ""
    term_3_ = row['back_words_original_exclude_word'][-3] if len(row['back_words_original_exclude_word']) >= 3 else ""

    terms.append(term_1_)
    terms.append(term_2_)
    terms.append(term_3_)

In [4]:
list(set(terms))

['',
 'sanguíneo',
 'volume',
 'psyllium',
 'concentração1',
 'arroz',
 'hipoalergênicofonte',
 'internação2',
 'animal',
 'estarcompre',
 'rice',
 '900g',
 'planta',
 'fracas',
 'hidrolisada',
 '70',
 'diferentes',
 '40g',
 'contendo',
 'glutamine',
 'gabriela',
 'biscoitos',
 'completa',
 'mascar',
 'lisa',
 '18g',
 '2802',
 'que',
 'ajuda',
 '7',
 'sonomelatonin',
 'corpo',
 'vegetal',
 'serotonina',
 'aminoácidos',
 'explosiva',
 'bovina',
 'biological',
 'energeticogo',
 'lactase',
 'científicos',
 'ntes',
 'porém',
 'venturini',
 'famoso',
 'feminina¹',
 'dutosproteínaswhey',
 'resveratrol',
 'potencializa',
 'health',
 'ineraisvitaminasômega',
 'rboidrato',
 'nutricional',
 'cuide',
 'dark',
 'bonita',
 'competições',
 'ôm',
 'mi',
 'vi',
 'inflamatória',
 'neutro',
 'acetilcisteína',
 'mental',
 'biotina',
 'protein100',
 'd3',
 'pele',
 'carboidrato',
 'benefits',
 'alimentação',
 'inteligente',
 'hipertrofia',
 'segundo',
 'ta',
 'leve',
 '0159',
 'ui',
 'pelos',
 'benefícios